In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
from bs4 import BeautifulSoup 


print('Libraries imported.')

Libraries imported.


In [2]:
source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
soup = BeautifulSoup(source,'lxml')

In [4]:
stat = soup.find('table', class_ ="wikitable sortable")
len(stat)
tr1 = stat.find_all('tr')

In [5]:
final = []
for row in tr1:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    #print(cols + ",")
    final = final + cols

In [6]:
s = [final[i:i+3] for i in range(0,len(final),3)]

In [7]:
d1 = pd.DataFrame()
d1['Postal Code'] = s
d1.head()

,Postal Code
0,"[M1A, Not assigned, Not assigned]"
1,"[M2A, Not assigned, Not assigned]"
2,"[M3A, North York, Parkwoods]"
3,"[M4A, North York, Victoria Village]"
4,"[M5A, Downtown Toronto, Regent Park, Harbourfr..."


In [8]:
d1.shape

(180, 1)

In [9]:
k = 0
for i in range(0,180):
        for j in range(0,1):
  #         print(j)
 #          print(final[k] + "," + final[k+1] + "," + final[k+2])
            d1['Postal Code'][i] = final[k] + "," + final[k+1] + "," + final[k+2]
            k = k + 3
            
         
d1.tail()

,Postal Code
175,"M5Z,Not assigned,Not assigned"
176,"M6Z,Not assigned,Not assigned"
177,"M7Z,Not assigned,Not assigned"
178,"M8Z,Etobicoke,Mimico NW, The Queensway West, S..."
179,"M9Z,Not assigned,Not assigned"


In [10]:
new = d1["Postal Code"].str.split(",", n = 2, expand = True) 
d1["a"]= new[0] 
d1["b"]= new[1] 
d1["c"]= new[2]
d1.head()

,Postal Code,a,b,c
0,"M1A,Not assigned,Not assigned",M1A,Not assigned,Not assigned
1,"M2A,Not assigned,Not assigned",M2A,Not assigned,Not assigned
2,"M3A,North York,Parkwoods",M3A,North York,Parkwoods
3,"M4A,North York,Victoria Village",M4A,North York,Victoria Village
4,"M5A,Downtown Toronto,Regent Park, Harbourfront",M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [11]:
d1 = d1.drop(['Postal Code'],axis=1)

In [12]:
d1.columns = ["Postal Code","Borough","Neighbourhood"]

In [13]:
d1 = d1[d1['Borough'] != 'Not assigned']

In [14]:
d1.shape

(103, 3)

In [15]:
cnt = 0 
for i in range(0,103):
    j = 2
    if d1.iloc[i][j] == 'Not assigned':
        d1.iloc[i][j] = d1.iloc[i][j-1]
        j = j + 1

In [16]:
d1 = d1.set_index('Postal Code')

In [17]:
d1.head()

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [18]:
d1 = d1.reset_index()

In [20]:
d1.shape

(103, 3)

# Part 2

In [21]:
cord = pd.read_csv("https://cocl.us/Geospatial_data")

In [22]:

cord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
d1.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
dp1 = pd.concat([d1, cord], axis=1)

In [25]:
dp1.head()

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476


In [26]:
dp1.shape

(103, 6)

# Part 3

In [27]:
toronto_data = dp1[dp1['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975


In [28]:
toronto_data.size

234

In [29]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                       

In [31]:
address = 'Toronto, CA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [33]:
# create map of New York using latitude and longitude values
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dp1['Latitude'], dp1['Longitude'], dp1['Borough'], dp1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [34]:
CLIENT_ID = 'G5VHDKWXQ54ZPOWQHS4QA4KLVY5EYWOXUWLMMF4ZV5GDFHKB' # your Foursquare ID
CLIENT_SECRET = 'SB3XYMHLNKHKJWE5MG02V3WRAFMEGIZJQF2X3VLQWZBOBHLN' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G5VHDKWXQ54ZPOWQHS4QA4KLVY5EYWOXUWLMMF4ZV5GDFHKB
CLIENT_SECRET:SB3XYMHLNKHKJWE5MG02V3WRAFMEGIZJQF2X3VLQWZBOBHLN


In [35]:
toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [36]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [37]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ee3b60b7f06d94cf8424f93'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough Village',
  'headerFullLocation': 'Scarborough Village, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 7,
  'suggestedBounds': {'ne': {'lat': 43.768072604500006,
    'lng': -79.18249216787879},
   'sw': {'lat': 43.7590725955, 'lng': -79.1949308321212}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4beee041e24d20a1cd857314',
       'name': 'RBC Royal Bank',
       'location': {'address': '4374 KINGSTON RD',
        'crossStreet': 'Kingston & Lawrence',
        'lat': 43.76678992471017,
        'lng': -79.19115118872593,
        '

In [38]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [39]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
3,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406
4,Woburn Medical Centre,Medical Center,43.766631,-79.192286


In [40]:
#Explore Beighbourhood in TORONTO

In [41]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                   'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [42]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )
toronto_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Regent Park, Harbourfront",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [43]:
toronto_venues.shape

(754, 7)

# Analyze Each Neighborhood

In [44]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
toronto_onehot.shape

(754, 207)

In [46]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.04878,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.06250,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,Central Bay Street,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [47]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                       venue  freq
0                  Cafeteria   1.0
1                Yoga Studio   0.0
2         Miscellaneous Shop   0.0
3                       Lake   0.0
4  Latin American Restaurant   0.0


----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2              Brewery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
               venue  freq
0        Pizza Place  0.25
1     Sandwich Place  0.25
2           Bus Line  0.25
3  Mobile Phone Shop  0.25
4        Yoga Studio  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Yoga Studio  0.06
1          Skate Park  0.06
2       Garden Center  0.06
3              Garden  0.06
4  Light Rail Station  0.06

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [49]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Cafeteria,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Bookstore,Pet Store
2,"Business reply mail Processing Centre, South C...",Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Auto Workshop,Park,Pizza Place,Butcher,Light Rail Station,Burrito Place,Restaurant,Brewery,Farmers Market
4,Central Bay Street,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


# Cluster Neighborhoods of TORONTO

In [53]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 3, 3, 3, 2, 3, 2, 0, 3], dtype=int32)

In [58]:
toronto_data.head()
#toronto_data.columns = ['Postal Code','Borough','Neighborhood','Postal Code','Latitude','Longitude','Cluster Labels']

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975


In [71]:
toronto_merged = toronto_data
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_



In [76]:
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,1
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476,3
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,3
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,3
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975,3


In [73]:
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.04878,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02439,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.06250,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0
4,Central Bay Street,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0


In [74]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_grouped = toronto_grouped.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [75]:
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,Cafeteria,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
1,"Brockton, Parkdale Village, Exhibition Place",0.02439,0.0,0.0,0.0,0.0,0.0,0.0,0.04878,0.0,...,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Bookstore,Pet Store
2,"Business reply mail Processing Centre, South C...",0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,Bus Line,Sandwich Place,Pizza Place,Mobile Phone Shop,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.06250,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,Yoga Studio,Auto Workshop,Park,Pizza Place,Butcher,Light Rail Station,Burrito Place,Restaurant,Brewery,Farmers Market
4,Central Bay Street,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,...,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [79]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

In [80]:
toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,1,Intersection,Breakfast Spot,Bank,Mexican Restaurant,Rental Car Location,Medical Center,Electronics Store,Women's Store,Diner,Dumpling Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476,3,Bakery,Hakka Restaurant,Bank,Gas Station,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Diner,Women's Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,3,College Stadium,General Entertainment,Café,Skating Rink,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,3,Chinese Restaurant,Electronics Store,Grocery Store,Breakfast Spot,Burger Joint,Fast Food Restaurant,Bank,Pharmacy,Gym Pool,Gym
4,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975,3,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [84]:
#Cluster-1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Downtown Toronto,-79.464763,0,Business Service,Airport,Snack Place,Park,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
10,Downtown Toronto,-79.318389,0,Curling Ice,Park,Skating Rink,Spa,Athletics & Sports,Pharmacy,Beer Store,Video Store,Convenience Store,Dim Sum Restaurant
28,West Toronto,-79.487262,0,Convenience Store,Pizza Place,Caribbean Restaurant,Women's Store,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [87]:
#Cluster 2 

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,-79.188711,1,Intersection,Breakfast Spot,Bank,Mexican Restaurant,Rental Car Location,Medical Center,Electronics Store,Women's Store,Diner,Dumpling Restaurant


In [89]:
# Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,-79.374714,2,Cafeteria,Women's Store,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
7,Downtown Toronto,-79.329656,2,Park,Fireworks Store,Food & Drink Shop,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
25,West Toronto,-79.422564,2,Grocery Store,Café,Park,Coffee Shop,Baby Store,Candy Store,Diner,Restaurant,Italian Restaurant,Nightclub
30,Downtown Toronto,-79.484450,2,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Pizza Place,Pub,Health Food Store,Dessert Shop,Bank,Bar
34,Downtown Toronto,-79.520999,2,Supplement Shop,Gym,Fast Food Restaurant,Kids Store,Burger Joint,Burrito Place,Discount Store,Social Club,Bakery,Hardware Store


In [90]:
#CLuster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,-79.239476,3,Bakery,Hakka Restaurant,Bank,Gas Station,Athletics & Sports,Caribbean Restaurant,Fried Chicken Joint,Thai Restaurant,Diner,Women's Store
2,Downtown Toronto,-79.264848,3,College Stadium,General Entertainment,Café,Skating Rink,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,Downtown Toronto,-79.318389,3,Chinese Restaurant,Electronics Store,Grocery Store,Breakfast Spot,Burger Joint,Fast Food Restaurant,Bank,Pharmacy,Gym Pool,Gym
4,East Toronto,-79.385975,3,Chinese Restaurant,Bank,Café,Japanese Restaurant,Women's Store,Dim Sum Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
6,Downtown Toronto,-79.442259,3,Coffee Shop,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant
9,West Toronto,-79.506944,3,Shopping Mall,Park,Grocery Store,Bank,Dessert Shop,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
12,East Toronto,-79.352188,3,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Ice Cream Shop,Restaurant,Furniture / Home Store,Trail,Liquor Store,Pub
13,Downtown Toronto,-79.315572,3,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop,Intersection,Restaurant,Brewery,Ice Cream Shop,Pub,Burrito Place,Pizza Place
14,West Toronto,-79.340923,3,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Middle Eastern Restaurant,Bookstore,Pet Store
15,East Toronto,-79.388790,3,Dessert Shop,Sandwich Place,Coffee Shop,Italian Restaurant,Pizza Place,Café,Sushi Restaurant,Gym,Gourmet Shop,Diner


In [91]:
# Cluster 5

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,West Toronto,-79.293031,4,Health Food Store,Trail,Pub,Women's Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
26,Central Toronto,-79.490074,4,Construction & Landscaping,Park,Trail,Basketball Court,Bakery,Women's Store,Dim Sum Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
